In [1]:
'''
Relacionar google drive con Collab, para poder usar mi drive como almacenamiento de archivos,
y como lugar donde van a reposar los outputs que se generen en este script.
'''
from google.colab import drive
drive.mount('/content/t3drive')
# Ejecutar el código anterior y aceptar lo solicitado; permisos y accesos

In [2]:
'''
Importación de paquetes módulos y funciones. Cabe aclarar que no es necesario importarlas
todos al inicio, solo que es una práctica heredada de la codificación tradicional que ayuda
a ser mas eficiente la compilación del código así como su lectura.
'''
import os # Para interactuar con el sistema operativo (rutas de archivos, crear directorios)
import pandas as pd # Para manipulación y análisis de datos (DataFrames)
import matplotlib.pyplot as plt # Para crear visualizaciones estáticas (gráficos)
import seaborn as sns # Para crear visualizaciones estadísticas atractivas, basado en matplotlib
from sklearn.model_selection import train_test_split # Para dividir datos en conjuntos de entrenamiento y prueba
from sklearn.linear_model import LogisticRegression # Implementación del modelo de Regresión Logística
from sklearn.metrics import accuracy_score, classification_report # Métricas para evaluar modelos de clasificación (precisión, informe de clasificación)
from sklearn.tree import DecisionTreeClassifier # Implementación del modelo de Árbol de Decisión
from sklearn.model_selection import GridSearchCV # Para ajuste de hiperparámetros utilizando búsqueda en cuadrícula y validación cruzada
from sklearn.neighbors import KNeighborsClassifier # Implementación del modelo k-NN (K-Nearest Neighbors)
from google.colab import files

In [3]:
'''
Definir las rutas para guardar las figuras, tablas y notebooks.
'''

# Definir las rutas base
base = '/content/t3drive/MyDrive/taller3'
figs = os.path.join(base, 'figures')
codes = os.path.join(base, 'codes')

# Crear las rutas sino existen
os.makedirs(figs, exist_ok=True)
os.makedirs(codes, exist_ok=True)

In [4]:
'''
Q1.1: Cargar los datos y convertirlos en un "Dataframe". Tener en cuenta que se encuentran en un archivo Excel.
'''

# Ruta completa hacia el los datos alojados en mi drive
file_path = os.path.join(base, "adult_data.xlsx")

# Leer la primera hoja del Excel (sheet 0)
import pandas as pd
df = pd.read_excel(file_path, sheet_name=0)

In [5]:
'''
Q1.2: Mostrar la forma del "Dataframe y mostrar primeros 5 registros.
'''
print("Forma del dataset:", df.shape)
display(df.head())

In [6]:
'''
Q1.3: Mostrar tipos de datos de cada columna
'''
print("Tipos de datos por columna:\n")
print(df.dtypes)

In [7]:
'''
Q1.4: Generar estadisticas descriptivas para las caracteristicas numericas con un formato adecuado.
'''
print("\nEstadísticas descriptivas para características numéricas:\n")

# Gnerar estadísticas descriptivas
descriptive_stats = df.describe()

# Aplicar formato al Dataframe
styled_stats = descriptive_stats.style.format("{:,.2f}")
display(styled_stats)

In [8]:
'''
Q1.5: Usar técnicas de visualización para explorar la distribución de características numéricas e
identificar posibles valores atípicos y guardar las figuras.
'''

# Seleccionar solo las columnas numéricas
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Iterar sobre las columnas numéricas y generar histogramas y boxplots
for col in numerical_cols:
    print(f"\nVisualizando la distribución de '{col}':")

    # Histograma
    plt.figure(figsize=(10, 5))
    sns.histplot(df[col], kde=True)
    plt.title(f'Histograma de {col}')
    plt.xlabel(col)
    plt.ylabel('Frecuencia')
    histogram_path = os.path.join(figs, f'histogram_{col}.png')
    plt.savefig(histogram_path, dpi=300, bbox_inches='tight')
    plt.show()

    # Boxplot (vertical)
    plt.figure(figsize=(5, 10)) # Ajustar el tamaño de la figura para la orientaciuón vertical
    sns.boxplot(y=df[col]) # Cambiar x a y para la orientación vertical
    plt.title(f'Boxplot de {col}')
    plt.ylabel(col) # Cambiar xlabel a ylabel
    boxplot_path = os.path.join(figs, f'boxplot_{col}.png')
    plt.savefig(boxplot_path, dpi=300, bbox_inches='tight')
    plt.show()

In [9]:
'''
Q1.6: Analizar la distribución de la variable objetivo "income".
'''

# Inspección previa porque parece que hay mas de dos categorías en esta variable
print("Etiquetas únicas crudas en 'income':")
print(sorted(df['income'].astype(str).unique()))

# Limpieza robusta de la columna 'income
# - pasa a str
# - strip de espacios
# - elimina puntos al final (caso '...K.')
# - normaliza mayúsculas/minúsculas todo a minusculas
# - colapsa espacios internos
income_clean = (df['income'].astype(str)
                .str.strip()
                .str.replace(r'\.$', '', regex=True)   # elimina punto final
                .str.replace(r'\s+', ' ', regex=True)  # colapsa espacios
                .str.lower())

# Mapeo a dos únicas clases canónicas
mapping = {
    '<=50k': '<=50K',
    '>50k': '>50K'
}
df['income'] = income_clean.map(mapping)

# Si hay valores inesperados (como '?'), los dejamos como NaN para no contaminar
unexpected = income_clean[~income_clean.isin(mapping.keys())].unique()
if len(unexpected) > 0:
    print("\nValores inesperados tras limpieza (se convierten a NaN):", sorted(unexpected))
    # forzar NaN donde no calce con mapping
    df.loc[~income_clean.isin(mapping.keys()), 'income'] = pd.NA

# Opcional: convertir a categórica con orden definido
df['income'] = pd.Categorical(df['income'], categories=['<=50K', '>50K'], ordered=True)

# Conteos finales
print("\nDistribución final de 'income':")
print(df['income'].value_counts(dropna=False))

# Histograma variable "income"
plt.figure(figsize=(7, 5))
sns.countplot(x='income', data=df, order=['<=50K', '>50K'])
plt.title('Distribución de la variable objetivo: income')
plt.xlabel('Income')
plt.ylabel('Conteo')
plt.tight_layout()

# Guardar figura en la carpeta definida
# Si el archivo ya existe, será reemplazado automáticamente)
plot_path = os.path.join(figs, "income_distribution.png")
plt.savefig(plot_path, dpi=300, bbox_inches='tight')

# Mostrar en pantalla
plt.show()

In [10]:
'''
Q2.1: Verificar valores nulos y reportar el conteo y porcentaje de datos faltantes para cada columna.
'''

# Verificar valores nulos
print("Conteo de valores nulos por columna:")
null_counts = df.isnull().sum()
display(null_counts)

# Calcular el porcentaje de valores nulos
print("\nPorcentaje de valores nulos por columna:")
null_percentages = (null_counts / len(df)) * 100
display(null_percentages)

In [11]:
'''
Q2.2: Verificar y eliminar filas duplicadas.
'''

# Verificar si hay filas duplicadas
print("Número de filas duplicadas antes de la eliminación:", df.duplicated().sum())

# Eliminar filas duplicadas
df.drop_duplicates(inplace=True)

# Verificar si hay filas duplicadas después de la eliminación
print("Número de filas duplicadas después de la eliminación:", df.duplicated().sum())

In [12]:
'''
Q2.3: Manejo de valores faltantes usando estrategias de imputación y eliminación
- Como se observo anteriormente, no hay valores nulos, por lo que no es necesario
hacer ninguna acción de las sugeridas.
'''

In [13]:
'''
Q2.4: Definir la variable objetivo (aunque la verdad es innecesario, se puede apuntar
a la columna income directamente).
'''
# La columna "income" es la variable objetivo para este análisis.
# Representa si el ingreso de un individuo es <=50K o >50K.
target_variable = 'income'
print(f"La variable objetivo definida es: {target_variable}")

In [14]:
'''
Q3.1: Dividir los datos en conjuntos de entrenamiento y prueba (proporción 80/20).
'''

# Definir las características (X) y la variable objetivo (y)
X = df.drop('income', axis=1)
y = df['income']

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
'''
El parámetro stratify=y asegura que la división de los datos en conjuntos de entrenamiento y
prueba mantenga aproximadamente la misma proporción de la variable objetivo (y) que la que
existe en el conjunto de datos original.
'''
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [15]:
'''
Q3.2: Verificar el balance de la variable objetivo en los conjuntos de entrenamiento
y prueba.
Esto ya se garantizó un poco con el paso anterior, sin embargo, se puede confirmar
realizando algunas impresiones.
'''

# Verificar la proporción de la variable objetivo en el conjunto de entrenamiento:
print("Proporción de la variable objetivo en el conjunto de entrenamiento:")
display(y_train.value_counts(normalize=True))

# Verificar la proporción de la variable objetivo en el conjunto de prueba:
print("\nProporción de la variable objetivo en el conjunto de prueba:")
display(y_test.value_counts(normalize=True))

In [16]:
'''
Q3.3: Reportar el tamaño de los sets de entrenamiento y prueba.
'''
print("Forma del conjunto de entrenamiento (X_train):", X_train.shape)
print("Forma del conjunto de prueba (X_test):", X_test.shape)
print("Forma de las etiquetas de entrenamiento (y_train):", y_train.shape)
print("Forma de las etiquetas de prueba (y_test):", y_test.shape)

In [17]:
'''
Q4.1: Calcular y visualizar la matriz de correlación entre las características numéricas y
la variable objetivo, y guardar la figura.
'''

# Seleccionar solo las columnas numéricas del conjunto de entrenamiento
numerical_cols_train = X_train.select_dtypes(include=['int64', 'float64'])

# Incluir la variable objetivo 'income' (ya limpia con 2 categorías) en el DataFrame para calcular la correlación
# Asegurarnos de que 'income' sea numérica para el cálculo de correlación.
# Podemos mapear las categorías a números (por ejemplo, <=50K a 0 y >50K a 1).
income_numeric = y_train.map({'<=50K': 0, '>50K': 1})

# Concatenar las características numéricas y la variable objetivo numérica
df_train_corr = pd.concat([numerical_cols_train, income_numeric], axis=1)

# Calcular la matriz de correlación
# Excluir NaN que puedan haber surgido del mapeo (aunque con la limpieza previa no deberían haber)
correlation_matrix = df_train_corr.corr().dropna(axis=0, how='all').dropna(axis=1, how='all')


# Visualizar la matriz de correlación usando un heatmap con colores invertidos
plt.figure(figsize=(12, 10)) # Aumentar tamaño para incluir la nueva variable
# Usar 'coolwarm_r' para que el azul sea positivo y el rojo negativo
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm_r', fmt=".2f", linewidths=.5)
plt.title('Matriz de Correlación de Características Numéricas e Income')
plt.tight_layout() # Ajustar para evitar que las etiquetas se superpongan

# Definir la ruta para guardar la figura
correlation_plot_path = os.path.join(figs, 'correlation_matrix_numerical_income.png')

# Guardar la figura
plt.savefig(correlation_plot_path, dpi=300, bbox_inches='tight')

plt.show()

In [18]:
'''
Q4.2: Visualizar las relaciones entre características numéricas y la variable objetivo
usando scatter plots y guardar la figura.
'''

# Seleccionar solo las columnas numéricas
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Añadir la columna 'income' al DataFrame temporal para el pairplot
df_plot = df[numerical_cols].copy()
df_plot['income'] = df['income']

# Crear scatter plots por pares de características numéricas, diferenciando por 'income'
# Usar un subconjunto de columnas si hay muchas para evitar una matriz de plots demasiado grande
if len(numerical_cols) > 6: # Limit to a reasonable number of columns for pairplot
    print("Mostrando pairplot para un subconjunto de características numéricas debido a la cantidad.")
    cols_to_plot = numerical_cols[:6].tolist() + ['income']
    g = sns.pairplot(df_plot[cols_to_plot], hue='income', palette='viridis')
else:
    g = sns.pairplot(df_plot, hue='income', palette='viridis')

plt.suptitle('Scatter Plots de Características Numéricas por Nivel de Ingreso', y=1.02)
plt.tight_layout(rect=[0, 0, 1, 1.02]) # Adjust layout to prevent suptitle overlap

# Definir la ruta para guardar la figura
pairplot_path = os.path.join(figs, 'pairplot_numerical_income.png')

# Guardar la figura
# Use g.fig to save the figure generated by seaborn's pairplot
g.fig.savefig(pairplot_path, dpi=300)

plt.show()

In [19]:
'''
Q4.3: Con base en los hallazgos, seleccionar las tres variables discriminativas más prometedoras
y construir una colección de clasificadores basados ​​en reglas utilizando predictores por pares.
- De aceuerdo a lo que dejaron ver las anteriores gráficas, las variables elegidas fueron:
- age
- education-num
- hours-per-week
- capital-gain

Q4.4: Para cada par de predictores, implemente un clasificador basado en reglas y evalúe su desempeño
utilizando precisión, especificidad, sensibilidad y puntuación F1.

Q4.5: Reportar el clasificador con mejor rendimiento y sus métricas de rendimiento.
'''

# Variables seleccionadas basadas en análisis previos
selected_features = ['age', 'education-num', 'hours-per-week', 'capital-gain']
target = 'income'

# Diccionario para almacenar los clasificadores entrenados y su rendimiento
classifiers = {}

# Iterar a través de todos los pares posibles de las variables seleccionadas
for i in range(len(selected_features)):
    for j in range(i + 1, len(selected_features)):
        feature1 = selected_features[i]
        feature2 = selected_features[j]
        pair = (feature1, feature2)

        print(f"\nConstruyendo clasificador para las variables: {pair}")

        # Seleccionar las variables emparejadas para entrenamiento y prueba
        X_train_pair = X_train[[feature1, feature2]]
        X_test_pair = X_test[[feature1, feature2]]

        # Inicializar y entrenar un modelo simple de Regresión Logística (como proxy basado en reglas)
        # La Regresión Logística puede ser vista como el aprendizaje de un límite de decisión lineal, que puede interpretarse como una regla.
        '''
        Regresión Logística:
        ¿Qué es? Es un algoritmo de clasificación utilizado para predecir la probabilidad de que una instancia
        pertenezca a una clase particular.
        - ¿Para qué sirve? Se usa comúnmente para problemas de clasificación binaria (dos clases), como predecir
          si un correo es spam o no, o si un cliente comprará un producto.
        - ¿Cómo funciona? A diferencia de la regresión lineal que predice un valor continuo, la regresión logística
          utiliza una función sigmoide (o logística) para mapear cualquier valor de entrada a un valor entre 0 y 1.
          Este valor se interpreta como la probabilidad de pertenecer a la clase positiva. Si la probabilidad es mayor
          que un umbral (comúnmente 0.5), se clasifica como la clase positiva; de lo contrario, como la clase negativa.
        En esencia, busca encontrar un límite de decisión lineal que mejor separe las clases en el espacio de características.
        '''
        model = LogisticRegression(random_state=42)
        model.fit(X_train_pair, y_train)

        # Realizar predicciones en el conjunto de prueba
        y_pred = model.predict(X_test_pair)

        # Evaluar el clasificador
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)

        print(f"Precisión (Accuracy): {accuracy:.4f}")
        print("Informe de Clasificación:")
        print(report)

        # Almacenar el modelo y su rendimiento
        classifiers[pair] = {
            'model': model,
            'accuracy': accuracy,
            'report': report
        }

In [20]:
'''
Q5.1: Implementar un clasificador de Árbol de Decisión.
Este modelo se utilizará para predecir la variable objetivo 'income'.
Se entrenará con los datos de entrenamiento y se evaluará con los datos de prueba.

Q5.2: Entrenar el arbol de decisión con los datos de entrenamiento.
'''

# Inicializar el clasificador de Árbol de Decisión
# Se puede ajustar el parámetro max_depth para controlar la complejidad del árbol
'''
Árbol de Decisión:
¿Qué es? Es un modelo de aprendizaje supervisado que se utiliza tanto para problemas de clasificación como de regresión.
¿Para qué sirve? Crea un modelo de predicción construyendo una estructura de árbol basada en decisiones sobre las características de los datos.
¿Cómo funciona? El árbol se construye dividiendo el conjunto de datos en subconjuntos más pequeños basándose en el valor de una característica.
Este proceso se repite recursively en cada nodo hijo hasta que se cumple un criterio de parada (por ejemplo, la profundidad máxima del árbol,
el número mínimo de muestras en un nodo, etc.). Las hojas del árbol representan las clases de salida (para clasificación) o valores (para regresión).
'''

# --- Inicio de la modificación para manejar variables categóricas ---

# Preprocesar las características: Convertir variables categóricas a numéricas usando One-Hot Encoding
# Aplicar a los conjuntos de entrenamiento y prueba por separado para evitar fuga de datos
X_train_processed = pd.get_dummies(X_train)
X_test_processed = pd.get_dummies(X_test)

# Asegurarse de que los conjuntos de entrenamiento y prueba tengan las mismas columnas después del one-hot encoding
# Esto maneja casos donde una categoría puede aparecer en el test set pero no en el train set (o viceversa)
X_train_processed, X_test_processed = X_train_processed.align(X_test_processed, join='left', axis=1, fill_value=0)

# --- Fin de la modificación ---


dt_classifier = DecisionTreeClassifier(random_state=42)

# Entrenar el modelo con los datos de entrenamiento preprocesados
print("Entrenando el clasificador de Árbol de Decisión...")
dt_classifier.fit(X_train_processed, y_train)

# Realizar predicciones en el conjunto de prueba preprocesado
y_pred_dt = dt_classifier.predict(X_test_processed)

# Evaluar el clasificador
accuracy_dt = accuracy_score(y_test, y_pred_dt)
report_dt = classification_report(y_test, y_pred_dt)

print("\nEvaluación del clasificador de Árbol de Decisión:")
print(f"Precisión (Accuracy): {accuracy_dt:.4f}")
print("Informe de Clasificación:")
print(report_dt)

In [21]:
'''
Q5.3: Experimentar con diferentes hiperparámetros del Árbol de Decisión
utilizando GridSearchCV para encontrar la mejor combinación.

Q5.4: Evaluar el desempéño del mejor modelo encontrado en el conjunto de prueba.
'''

'''
Hiperparámetro:
En el contexto del aprendizaje automático, un hiperparámetro es un parámetro cuyo valor se utiliza para controlar el proceso de aprendizaje en sí mismo.
A diferencia de los parámetros del modelo (que se aprenden de los datos), los hiperparámetros se establecen antes de que comience el entrenamiento del modelo.
Ejemplos para un Árbol de Decisión incluyen: max_depth, min_samples_split, criterion.
Ajustar los hiperparámetros es crucial para optimizar el rendimiento del modelo y evitar el sobreajuste o subajuste.
'''

'''
GridSearchCV:
Es una técnica de optimización de hiperparámetros proporcionada por la librería scikit-learn.
Funciona realizando una búsqueda exhaustiva sobre un conjunto predefinido de valores de hiperparámetros (una "cuadrícula").
Para cada combinación de hiperparámetros en la cuadrícula, entrena y evalúa el modelo utilizando validación cruzada (cv).
Finalmente, selecciona la combinación de hiperparámetros que obtuvo la mejor puntuación promedio (definida por el parámetro 'scoring') durante la validación cruzada.
Es una forma sistemática de encontrar la mejor configuración de hiperparámetros dentro de un espacio de búsqueda definido.
'''

# Definir el espacio de búsqueda de hiperparámetros
# Experimentaremos con 'max_depth' y 'min_samples_split'
param_grid = {
    'max_depth': [None, 5, 10, 15, 20], # None significa sin límite de profundidad
    'min_samples_split': [2, 5, 10, 20]
}

# Inicializar el clasificador de Árbol de Decisión
dt_classifier = DecisionTreeClassifier(random_state=42)

# Inicializar GridSearchCV
# 'estimator' es el modelo que queremos optimizar (DecisionTreeClassifier)
# 'param_grid' es el diccionario con los hiperparámetros a probar y sus valores
# 'cv' especifica el número de folds para la validación cruzada (por ejemplo, 5)
# 'scoring' especifica la métrica a optimizar (por ejemplo, 'accuracy')
# 'n_jobs=-1' usa todos los procesadores disponibles, acelerando la búsqueda
grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

print("Iniciando búsqueda de hiperparámetros con GridSearchCV...")

# Realizar la búsqueda en los datos de entrenamiento preprocesados
# Usamos los datos preprocesados (X_train_processed) que incluyen el one-hot encoding
grid_search.fit(X_train_processed, y_train)

print("Búsqueda de hiperparámetros completada.")

# Mostrar los mejores hiperparámetros encontrados
print("\nMejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Mostrar la mejor puntuación (accuracy) obtenida con esos hiperparámetros
print("\nMejor Accuracy (validación cruzada):")
print(f"{grid_search.best_score_:.4f}")

# Evaluar el mejor modelo encontrado en el conjunto de prueba
best_dt_model = grid_search.best_estimator_
y_pred_best_dt = best_dt_model.predict(X_test_processed)

print("\nEvaluación del mejor modelo en el conjunto de prueba:")
print(f"Precisión (Accuracy) en el conjunto de prueba: {accuracy_score(y_test, y_pred_best_dt):.4f}")
print("Informe de Clasificación en el conjunto de prueba:")
print(classification_report(y_test, y_pred_best_dt))

In [22]:
'''
Q5.5: Seleccionar e identificar la configuración con mejor rendimiento
en función de las métricas e informe de los hiperparámetros elegidos.
'''

# Los resultados de la búsqueda de hiperparámetros se almacenan en el objeto grid_search
# que se definió y ejecutó en la celda anterior.

# La mejor combinación de hiperparámetros se encuentra en .best_params_
print("La mejor configuración de hiperparámetros encontrada es:")
display(grid_search.best_params_)

# La mejor puntuación (accuracy) obtenida con esta configuración durante la validación cruzada
print("\nLa mejor precisión (Accuracy) durante la validación cruzada fue:")
print(f"{grid_search.best_score_:.4f}")

# Para ver el rendimiento del modelo con los mejores hiperparámetros en el conjunto de prueba,
# ya lo calculamos y mostramos al final de la celda anterior (Q5.3).
# Podemos volver a imprimirlo aquí para consolidar los resultados.

# Acceder al mejor estimador (el modelo con los mejores hiperparámetros)
best_dt_model = grid_search.best_estimator_

# Realizar predicciones en el conjunto de prueba usando el mejor modelo
y_pred_best_dt = best_dt_model.predict(X_test_processed)

# Evaluar el mejor modelo en el conjunto de prueba
accuracy_best_test = accuracy_score(y_test, y_pred_best_dt)
report_best_test = classification_report(y_test, y_pred_best_dt)

print("\nEvaluación del mejor modelo en el conjunto de prueba:")
print(f"Precisión (Accuracy) en el conjunto de prueba: {accuracy_best_test:.4f}")
print("Informe de Clasificación en el conjunto de prueba:")
print(report_best_test)

In [23]:
'''
Q6.1: El proceso 'One Hot Encoding' ya se realizó anteriormente
Q6.2,3: Implementar un clasificador k-NN (K-Nearest Neighbors).
Este modelo se utilizará para predecir la variable objetivo 'income'.
Se entrenará con los datos de entrenamiento preprocesados y se evaluará
con los datos de prueba preprocesados.
'''

# Inicializar el clasificador k-NN
# Puedes ajustar el número de vecinos (n_neighbors)
'''
K-Nearest Neighbors (k-NN):
- ¿Qué es? Es un algoritmo de clasificación y regresión no paramétrico simple.
- ¿Para qué sirve? Se utiliza para predecir la clase de un punto de datos basándose en las clases de sus 'k' vecinos
  más cercanos en el espacio de características.
- ¿Cómo funciona? Para clasificar un nuevo punto, el algoritmo calcula su distancia a todos los puntos en el conjunto de entrenamiento.
  Luego, selecciona los 'k' puntos más cercanos. La clase del nuevo punto se asigna por la mayoría de votos de sus 'k' vecinos más cercanos.
  La elección del valor de 'k' es crucial.
'''
knn_classifier = KNeighborsClassifier(n_neighbors=5) # Usamos un valor inicial de k=5

# Entrenar el modelo con los datos de entrenamiento preprocesados
print("Entrenando el clasificador k-NN...")
# Usamos los datos ya codificados
knn_classifier.fit(X_train_processed, y_train)

# Realizar predicciones en el conjunto de prueba preprocesado
print("Realizando predicciones con el clasificador k-NN...")
y_pred_knn = knn_classifier.predict(X_test_processed)

# Evaluar el clasificador
accuracy_knn = accuracy_score(y_test, y_pred_knn)
report_knn = classification_report(y_test, y_pred_knn)

print("\nEvaluación del clasificador k-NN:")
print(f"Precisión (Accuracy): {accuracy_knn:.4f}")
print("Informe de Clasificación:")
print(report_knn)

In [24]:
'''
Q6.4: Experimentar con diferentes valores de k para el clasificador k-NN
y evaluar el rendimiento en el conjunto de prueba.
Q6.5 Informar el rendimiento para diferentes valores de k e identificar el k óptimo según las métricas.
'''

# Definir un rango de valores para k a experimentar
# Puedes ajustar este rango según sea necesario
k_values = range(1, 31) # Experimentaremos con k desde 1 hasta 30

# Lista para almacenar la precisión para cada valor de k
accuracy_scores = []

print("Experimentando con diferentes valores de k para el clasificador k-NN...")

# Iterar sobre los valores de k
for k in k_values:
    # Inicializar el clasificador k-NN con el valor actual de k
    knn_classifier = KNeighborsClassifier(n_neighbors=k)

    # Entrenar el modelo con los datos de entrenamiento preprocesados
    # Usamos los datos ya codificados (X_train_processed)
    knn_classifier.fit(X_train_processed, y_train)

    # Realizar predicciones en el conjunto de prueba preprocesado
    y_pred_knn = knn_classifier.predict(X_test_processed)

    # Evaluar el clasificador y almacenar la precisión
    accuracy = accuracy_score(y_test, y_pred_knn)
    accuracy_scores.append(accuracy)

    print(f"k = {k}: Precisión (Accuracy) = {accuracy:.4f}")

# Visualizar la precisión en función de k
plt.figure(figsize=(12, 6))
plt.plot(k_values, accuracy_scores, marker='o', linestyle='-')
plt.title('Precisión del Clasificador k-NN vs. Valor de k')
plt.xlabel('Valor de k (Número de Vecinos)')
plt.ylabel('Precisión (Accuracy)')
plt.xticks(k_values[::2]) # Mostrar etiquetas solo para algunos valores de k
plt.grid(True)
plt.tight_layout() # Ajustar el diseño

# Definir la ruta y el nombre del archivo para guardar la figura
# Usaremos la variable 'figs' definida anteriormente para la carpeta de figuras
knn_k_plot_path = os.path.join(figs, 'knn_accuracy_vs_k.png')

# Guardar la figura en la ruta especificada
plt.savefig(knn_k_plot_path, dpi=300, bbox_inches='tight') # dpi=300 para buena resolución

plt.show()

# Identificar el mejor valor de k y su precisión
best_k_index = accuracy_scores.index(max(accuracy_scores))
best_k = k_values[best_k_index]
best_accuracy = accuracy_scores[best_k_index]

print(f"\nEl mejor valor de k encontrado es {best_k} con una precisión de {best_accuracy:.4f}")
print(f"Figura de Precisión vs k guardada en: {knn_k_plot_path}")

In [25]:
'''
Guardar en mi drive los notebooks en formato .ipynb y .py.
'''

notebook_name = "notebook_taller3"

# Ruta completa de salida en Drive
ipynb_path = os.path.join(codes, f"{notebook_name}.ipynb")
py_path = os.path.join(codes, f"{notebook_name}.py")

# Ruta temporal para guardar el notebook actual antes de convertir
temp_ipynb_path = "/content/temp_notebook.ipynb"

# Guardar el notebook actual a una ruta temporal usando el comando mágico %notebook
# Esto asegura que nbconvert tenga un archivo .ipynb específico para trabajar
get_ipython().run_line_magic('notebook', temp_ipynb_path)


# Guardar en Drive en formato .ipynb
# Usar el archivo temporal guardado por %notebook
!jupyter nbconvert --to notebook --output "{ipynb_path}" "{temp_ipynb_path}"

# Guardar en Drive en formato .py
# Usar el archivo temporal guardado por %notebook
!jupyter nbconvert --to script   --output "{py_path}"    "{temp_ipynb_path}"

# Opcional: Eliminar el archivo temporal después de la conversión
# import os
# os.remove(temp_ipynb_path)

print(f"Archivos guardados en Drive:\n- {ipynb_path}\n- {py_path}")

In [26]:
'''
Q7.1: Construir un clasificador de votación mayoritaria que combine las predicciones
de los tres clasificadores con mejor rendimiento encontrados hasta ahora.

Los tres clasificadores con mejor rendimiento identificados son:
1. Árbol de Decisión con hiperparámetros optimizados (max_depth=10, min_samples_split=20)
2. k-NN con k óptimo (k=25, basado en la exploración)
3. El mejor clasificador basado en reglas de los pares de variables (educacion-num, capital-gain)

Para el clasificador basado en reglas, usamos una Regresión Logística como proxy.

'''

from sklearn.ensemble import VotingClassifier

print("Construyendo y evaluando el clasificador de votación mayoritaria...")

# --- 1. Entrenar/obtener el mejor Árbol de Decisión (ya entrenado en Q5.3) ---
# best_dt_model = grid_search.best_estimator_ # Ya tenemos este modelo de la celda Q5.3

# --- 2. Entrenar el k-NN con el mejor k encontrado (k=25) ---
best_knn_model = KNeighborsClassifier(n_neighbors=best_k)
best_knn_model.fit(X_train_processed, y_train)


# --- 3. Entrenar el mejor clasificador basado en reglas (Regresión Logística con 'education-num' y 'capital-gain') ---
# Identificar las variables del mejor clasificador basado en reglas
best_rule_pair = ('education-num', 'capital-gain')

# Seleccionar las variables correspondientes para entrenamiento y prueba
X_train_rule = X_train_processed[list(best_rule_pair)] # Usar processed para consistencia
X_test_rule = X_test_processed[list(best_rule_pair)]   # Usar processed para consistencia

# Entrenar el modelo de Regresión Logística
best_rule_model = LogisticRegression(random_state=42)
best_rule_model.fit(X_train_rule, y_train)


# --- Construir el clasificador de votación mayoritaria ---
# Definir los estimadores a incluir en el VotingClassifier
# Los nombres ('dt', 'knn', 'rule') son etiquetas para identificar los modelos
estimators = [
    ('dt', best_dt_model),
    ('knn', best_knn_model),
    ('rule', best_rule_model)
]

# Inicializar el VotingClassifier
# 'voting='hard'' significa votación mayoritaria (la clase predicha es la que recibe más votos)
# 'n_jobs=-1' usa todos los procesadores disponibles
ensemble_classifier = VotingClassifier(estimators=estimators, voting='hard', n_jobs=-1)

# Entrenar el clasificador de conjunto con los datos de entrenamiento preprocesados
# El VotingClassifier entrenará cada modelo individualmente
print("Entrenando el clasificador de conjunto (VotingClassifier)...")
# Usar los datos preprocesados, que son necesarios para DT y k-NN,
# y las columnas relevantes para el modelo de reglas se seleccionarán internamente si se pasa X_train_processed completo.
# Opcionalmente, podrías crear un Pipeline con preprocesamiento, pero para simplicidad aquí
# asumimos que los modelos individuales saben cómo manejar las columnas relevantes de X_train_processed.
# Sin embargo, dado que el modelo 'rule' solo usa dos columnas, debemos asegurar que reciba solo esas.
# Una forma robusta es pasar X_train_processed completo y el modelo 'rule' debe manejarlo.
# Vamos a asegurar que los modelos en el ensemble trabajen con las mismas features que fueron entrenados individualmente.
# Esto implica que si un modelo individual fue entrenado con X_train_processed, el ensemble debe recibir X_train_processed.

# Es crucial que todos los modelos en el ensemble se entrenen y predigan usando el mismo conjunto de características.
# Como best_dt_model y best_knn_model fueron entrenados con X_train_processed, el ensemble debe ser entrenado con X_train_processed.
# El modelo 'rule' también debe ser compatible con la forma de X_train_processed o se debe usar un pipeline.
# Dado que LogisticRegression puede manejar subconjuntos de columnas, y fue entrenado con un subconjunto,
# podríamos tener un problema aquí si el ensemble pasa el X_train_processed completo.
# Para ser más rigurosos, deberíamos re-entrenar el modelo de reglas dentro del ensemble o usar un pipeline.
# Por ahora, para simplificar y mantener la estructura actual, re-entrenaremos los modelos en el ensemble.

# Re-entrenar los modelos individuales con los datos preprocesados antes de construir el ensemble
# (Aunque best_dt_model y best_knn_model ya lo están, esto asegura consistencia si se ejecuta esta celda sola)

# Re-entrenar best_dt_model
best_dt_model_ensemble = DecisionTreeClassifier(max_depth=grid_search.best_params_['max_depth'],
                                                min_samples_split=grid_search.best_params_['min_samples_split'],
                                                random_state=42)
best_dt_model_ensemble.fit(X_train_processed, y_train)

# Re-entrenar best_knn_model
best_knn_model_ensemble = KNeighborsClassifier(n_neighbors=best_k)
best_knn_model_ensemble.fit(X_train_processed, y_train)

# Re-entrenar best_rule_model (Regresión Logística) con el subconjunto de columnas
best_rule_model_ensemble = LogisticRegression(random_state=42)
best_rule_model_ensemble.fit(X_train_processed[list(best_rule_pair)], y_train) # Entrenar solo con las columnas relevantes


# Actualizar la lista de estimadores con los modelos re-entrenados
estimators_ensemble = [
    ('dt', best_dt_model_ensemble),
    ('knn', best_knn_model_ensemble),
    ('rule', best_rule_model_ensemble)
]

# Inicializar el VotingClassifier con los modelos re-entrenados
ensemble_classifier = VotingClassifier(estimators=estimators_ensemble, voting='hard', n_jobs=-1)

# Entrenar el clasificador de conjunto
# Aquí es donde está el detalle: el VotingClassifier espera X_train y X_test con las mismas columnas
# para todos los estimadores, a menos que se usen pipelines.
# Dado que nuestro modelo 'rule' fue entrenado solo con dos columnas, pasar X_train_processed completo fallará.
# La forma más correcta es usar Pipelines o asegurar que todos los modelos manejen el mismo input.

# --- Estrategia Corregida: Crear Pipelines para cada modelo si necesitan preprocesamiento específico ---
# Sin embargo, dado que X_train_processed ya contiene todas las columnas one-hot encoded,
# los modelos DT y k-NN pueden usarlas directamente. El modelo de reglas (LR) solo necesita 2 de esas columnas.

# Opción 1: Modificar el modelo de reglas para que pueda aceptar X_train_processed completo y solo usar las columnas que necesita.
# Esto requeriría un wrapper o un Pipeline.

# Opción 2: Entrenar el VotingClassifier con X_train_processed completo, asumiendo que cada estimador
# puede seleccionar las columnas que necesita. Esto NO es el comportamiento por defecto de sklearn estimators.

# Opción 3 (La más robusta en sklearn): Usar ColumnTransformer dentro de un Pipeline si los modelos individuales
# necesitan diferentes subconjuntos de columnas o preprocesamientos. Esto es más complejo.

# --- Volvamos a la estrategia simple pero asegurándonos de que la entrada al VotingClassifier sea consistente ---
# Los modelos DT y k-NN usan X_train_processed. El modelo de reglas usa X_train_processed[list(best_rule_pair)].
# El VotingClassifier requiere que todos los estimadores en 'estimators' acepten el mismo input (X_train_processed).
# El modelo de reglas actual (LogisticRegression) no aceptará X_train_processed completo porque fue entrenado solo con 2 columnas.

# Para que funcione, el modelo de reglas dentro del VotingClassifier *debe* aceptar X_train_processed como input.
# Una forma de lograr esto sin Pipelines complejos es crear un "adaptador" o simplemente
# asegurar que el modelo de reglas dentro del ensemble pueda manejar el input completo.
# La forma más sencilla aquí, dado el contexto, es re-definir el modelo de reglas para que acepte todas las columnas
# pero solo use las que necesita. Esto generalmente se hace con un Pipeline que selecciona las columnas primero.

# --- Vamos a implementar una versión simplificada usando una función que crea el modelo de reglas con selección de columnas ---
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Crear un Pipeline para el modelo de reglas que primero selecciona las columnas
rule_pipeline = Pipeline([
    ('selector', ColumnTransformer([('passthrough', 'passthrough', list(best_rule_pair))],
                                  remainder='drop')), # Selecciona solo las columnas del par
    ('model', LogisticRegression(random_state=42))   # Aplica la regresión logística
])

# Ahora, este rule_pipeline puede aceptar X_train_processed completo y solo usará las columnas correctas.

# Actualizar la lista de estimadores con el pipeline de reglas
estimators_ensemble_piped = [
    ('dt', best_dt_model_ensemble), # DT ya entrenado con X_train_processed
    ('knn', best_knn_model_ensemble), # k-NN ya entrenado con X_train_processed
    ('rule_pipe', rule_pipeline)      # Pipeline de reglas que maneja X_train_processed
]

# Inicializar el VotingClassifier con los modelos y el pipeline de reglas
ensemble_classifier = VotingClassifier(estimators=estimators_ensemble_piped, voting='hard', n_jobs=-1)

# Entrenar el clasificador de conjunto con los datos de entrenamiento preprocesados
# Ahora sí, pasamos X_train_processed completo al ensemble_classifier
print("Entrenando el clasificador de conjunto (VotingClassifier) con Pipelines...")
ensemble_classifier.fit(X_train_processed, y_train)

print("Entrenamiento del clasificador de conjunto completado.")


# Realizar predicciones en el conjunto de prueba preprocesado
print("Realizando predicciones con el clasificador de votación mayoritaria...")
y_pred_ensemble = ensemble_classifier.predict(X_test_processed)

# Evaluar el clasificador de conjunto
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
report_ensemble = classification_report(y_test, y_pred_ensemble)

print("\nEvaluación del clasificador de votación mayoritaria:")
print(f"Precisión (Accuracy) en el conjunto de prueba: {accuracy_ensemble:.4f}")
print("Informe de Clasificación en el conjunto de prueba:")
print(report_ensemble)

# Opcional: Comparar con los modelos individuales (ya lo tenemos de celdas anteriores)
print("\nComparación con los modelos individuales:")
print(f"- Árbol de Decisión (Optimizado): Precisión = {accuracy_best_test:.4f}")
print(f"- k-NN (Mejor k={best_k}): Precisión = {best_accuracy:.4f} (Note: Esta es la precisión en el test set del modelo individual, no del cross-validation)")
# Para el modelo de reglas individual, recuperamos su precisión del diccionario classifiers
best_rule_accuracy_individual = classifiers[best_rule_pair]['accuracy']
print(f"- Regresión Logística (Pair: {best_rule_pair}): Precisión = {best_rule_accuracy_individual:.4f}")

In [27]:
# Define the different test sizes for the train-test splits.
# This corresponds to train sizes of 60%, 40%, 20%, and 10% respectively.
split_ratios = [0.6, 0.4, 0.2, 0.1]

print("Different train-test split ratios (represented by test size):")
print(split_ratios)

In [28]:
# Initialize a dictionary to store the performance metrics for each split ratio.
results = {}

# Start a loop that iterates through each value in the split_ratios list.
for test_size in split_ratios:
    # Print a message indicating the current split ratio being processed.
    print(f"\nProcessing split ratio (test size): {test_size}")

In [29]:
    # Split the data into training and testing sets based on the current split ratio.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

    # Preprocess the characteristics: Convert variables categorical to numerical using One-Hot Encoding
    # Apply to the training and test sets separately to avoid data leakage
    X_train_processed = pd.get_dummies(X_train)
    X_test_processed = pd.get_dummies(X_test)

    # Ensure that the training and test sets have the same columns after one-hot encoding
    # This handles cases where a category may appear in the test set but not in the training set (or vice versa)
    X_train_processed, X_test_processed = X_train_processed.align(X_test_processed, join='left', axis=1, fill_value=0)

    print(f"  Data split and preprocessed for test size {test_size}.")
    print(f"  X_train_processed shape: {X_train_processed.shape}")
    print(f"  X_test_processed shape: {X_test_processed.shape}")

In [30]:
    # Implement and evaluate the Decision Tree classifier.
    print("  Training and evaluating Decision Tree...")
    dt_classifier = DecisionTreeClassifier(random_state=42)
    dt_classifier.fit(X_train_processed, y_train)
    y_pred_dt = dt_classifier.predict(X_test_processed)
    accuracy_dt = accuracy_score(y_test, y_pred_dt)
    report_dt = classification_report(y_test, y_pred_dt, output_dict=True)
    print(f"    Decision Tree Accuracy: {accuracy_dt:.4f}")

    # Store Decision Tree results
    if test_size not in results:
        results[test_size] = {}
    results[test_size]['Decision Tree'] = {
        'accuracy': accuracy_dt,
        'report': report_dt
    }

In [31]:
    # Implement and evaluate the k-NN classifier.
    print("  Training and evaluating k-NN...")
    # Using the best k found in the previous single split analysis (k=25)
    knn_classifier = KNeighborsClassifier(n_neighbors=best_k)
    knn_classifier.fit(X_train_processed, y_train)
    y_pred_knn = knn_classifier.predict(X_test_processed)
    accuracy_knn = accuracy_score(y_test, y_pred_knn)
    report_knn = classification_report(y_test, y_pred_knn, output_dict=True)
    print(f"    k-NN Accuracy: {accuracy_knn:.4f}")

    # Store k-NN results
    results[test_size]['k-NN'] = {
        'accuracy': accuracy_knn,
        'report': report_knn
    }

In [32]:
# Select the features for the Logistic Regression model
best_rule_pair = ('education-num', 'capital-gain')
X_train_rule = X_train_processed[list(best_rule_pair)]
X_test_rule = X_test_processed[list(best_rule_pair)]

# Initialize and train the Logistic Regression model
print("  Training and evaluating Logistic Regression (on selected features)...")
lr_classifier = LogisticRegression(random_state=42)
lr_classifier.fit(X_train_rule, y_train)

# Make predictions on the test set
y_pred_lr = lr_classifier.predict(X_test_rule)

# Evaluate the classifier
accuracy_lr = accuracy_score(y_test, y_pred_lr)
report_lr = classification_report(y_test, y_pred_lr, output_dict=True)
print(f"    Logistic Regression Accuracy: {accuracy_lr:.4f}")

# Store Logistic Regression results
results[test_size]['Logistic Regression'] = {
    'accuracy': accuracy_lr,
    'report': report_lr
}

In [33]:
# Create a list of estimators for the Voting Classifier.
# Use the already trained models from the current split iteration.
estimators = [
    ('dt', dt_classifier),
    ('knn', knn_classifier),
    ('lr', lr_classifier) # Use the LR classifier trained on selected features
]

# Instantiate a VotingClassifier object with 'hard' voting.
# n_jobs=-1 uses all available CPU cores for parallel processing, which can speed up training
ensemble_classifier = VotingClassifier(estimators=estimators, voting='hard', n_jobs=-1)

# Train the VotingClassifier on the preprocessed training data.
print("  Training Voting Classifier...")
ensemble_classifier.fit(X_train_processed, y_train)
print("  Voting Classifier trained.")

In [34]:
# Make predictions on the preprocessed test data using the trained Voting Classifier.
print("  Evaluating Voting Classifier...")
y_pred_ensemble = ensemble_classifier.predict(X_test_processed)

# Evaluate the performance of the Voting Classifier.
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
report_ensemble = classification_report(y_test, y_pred_ensemble, output_dict=True)

print(f"    Voting Classifier Accuracy: {accuracy_ensemble:.4f}")

# Store the performance metrics of the Voting Classifier in the results dictionary.
results[test_size]['Voting Classifier'] = {
    'accuracy': accuracy_ensemble,
    'report': report_ensemble
}
print("  Voting Classifier evaluated and results stored.")

In [35]:
import pandas as pd

# Create a dictionary to hold accuracies for easier DataFrame creation
accuracy_summary = {}
for test_size, models in results.items():
    accuracy_summary[test_size] = {model_name: model_results['accuracy'] for model_name, model_results in models.items()}

# Create a DataFrame from the accuracy summary dictionary
# Transpose the DataFrame so models are rows and test sizes are columns
accuracy_df = pd.DataFrame.from_dict(accuracy_summary, orient='index').transpose()

# Sort columns by test size (ascending) for better readability
accuracy_df = accuracy_df.sort_index(axis=1)

print("Accuracy Summary Across Different Train-Test Splits:")
display(accuracy_df)

In [36]:
import matplotlib.pyplot as plt
import seaborn as sns
import os

# (Optional) Create a line plot showing the accuracy of each model across the different split ratios.
plt.figure(figsize=(12, 7))
sns.lineplot(data=accuracy_df.transpose(), marker='o') # Transpose back for plotting test_size on x-axis
plt.title('Model Accuracy vs. Train-Test Split Ratio')
plt.xlabel('Test Size (Split Ratio)')
plt.ylabel('Accuracy')
plt.xticks(accuracy_df.columns) # Set x-ticks to the test sizes
plt.grid(True)
plt.legend(title='Model')
plt.tight_layout() # Adjust layout

# (Optional) Save the plot to the figures directory defined by the figs variable.
plot_path = os.path.join(figs, "model_accuracy_vs_split_ratio.png")
plt.savefig(plot_path, dpi=300)
print(f"Accuracy plot saved to: {plot_path}")

# (Optional) Display the plot.
plt.show()

In [37]:
# Initialize a dictionary to store the performance metrics for each split ratio.
results = {}

# Start a loop that iterates through each value in the split_ratios list.
for test_size in split_ratios:
    # Print a message indicating the current split ratio being processed.
    print(f"\nProcesando proporción de división (tamaño del conjunto de prueba): {test_size}")

    # Split the data into training and testing sets based on the current split ratio.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

    # Preprocesar las características: Convertir variables categóricas a numéricas usando One-Hot Encoding
    # Aplicar a los conjuntos de entrenamiento y prueba por separado para evitar fuga de datos
    X_train_processed = pd.get_dummies(X_train)
    X_test_processed = pd.get_dummies(X_test)

    # Asegurarse de que los conjuntos de entrenamiento y prueba tengan las mismas columnas después del one-hot encoding
    # Esto maneja casos donde una categoría puede aparecer en el test set pero no en el training set (o viceversa)
    X_train_processed, X_test_processed = X_train_processed.align(X_test_processed, join='left', axis=1, fill_value=0)

    print(f"  Datos divididos y preprocesados para tamaño de prueba {test_size}.")
    print(f"  Forma de X_train_processed: {X_train_processed.shape}")
    print(f"  Forma de X_test_processed: {X_test_processed.shape}")

    # Implementar y evaluar el clasificador de Árbol de Decisión.
    print("  Entrenando y evaluando Árbol de Decisión...")
    dt_classifier = DecisionTreeClassifier(random_state=42)
    dt_classifier.fit(X_train_processed, y_train)
    y_pred_dt = dt_classifier.predict(X_test_processed)
    accuracy_dt = accuracy_score(y_test, y_pred_dt)
    report_dt = classification_report(y_test, y_pred_dt, output_dict=True)
    print(f"    Precisión del Árbol de Decisión: {accuracy_dt:.4f}")

    # Almacenar resultados del Árbol de Decisión
    if test_size not in results:
        results[test_size] = {}
    results[test_size]['Decision Tree'] = {
        'accuracy': accuracy_dt,
        'report': report_dt
    }

    # Implementar y evaluar el clasificador k-NN.
    print("  Entrenando y evaluando k-NN...")
    # Usando el mejor k encontrado en el análisis de división única anterior (k=25)
    knn_classifier = KNeighborsClassifier(n_neighbors=best_k)
    knn_classifier.fit(X_train_processed, y_train)
    y_pred_knn = knn_classifier.predict(X_test_processed)
    accuracy_knn = accuracy_score(y_test, y_pred_knn)
    report_knn = classification_report(y_test, y_pred_knn, output_dict=True)
    print(f"    Precisión de k-NN: {accuracy_knn:.4f}")

    # Almacenar resultados de k-NN
    results[test_size]['k-NN'] = {
        'accuracy': accuracy_knn,
        'report': report_knn
    }

    # Seleccionar las características para el modelo de Regresión Logística
    best_rule_pair = ('education-num', 'capital-gain')
    X_train_rule = X_train_processed[list(best_rule_pair)]
    X_test_rule = X_test_processed[list(best_rule_pair)]

    # Inicializar y entrenar el modelo de Regresión Logística
    print("  Entrenando y evaluando Regresión Logística (en características seleccionadas)...")
    lr_classifier = LogisticRegression(random_state=42)
    lr_classifier.fit(X_train_rule, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_pred_lr = lr_classifier.predict(X_test_rule)

    # Evaluar el clasificador
    accuracy_lr = accuracy_score(y_test, y_pred_lr)
    report_lr = classification_report(y_test, y_pred_lr, output_dict=True)
    print(f"    Precisión de Regresión Logística: {accuracy_lr:.4f}")

    # Almacenar resultados de Regresión Logística
    results[test_size]['Logistic Regression'] = {
        'accuracy': accuracy_lr,
        'report': report_lr
    }

    # Crear una lista de estimadores para el Voting Classifier.
    # Usar los modelos ya entrenados de la iteración de división actual.
    estimators = [
        ('dt', dt_classifier),
        ('knn', knn_classifier),
        ('lr', lr_classifier) # Usar el clasificador LR entrenado en características seleccionadas
    ]

    # Instanciar un objeto VotingClassifier con votación 'hard'.
    # n_jobs=-1 usa todos los núcleos de CPU disponibles para procesamiento paralelo, lo que puede acelerar el entrenamiento
    ensemble_classifier = VotingClassifier(estimators=estimators, voting='hard', n_jobs=-1)

    # Entrenar el VotingClassifier en los datos de entrenamiento preprocesados.
    print("  Entrenando Clasificador de Votación...")
    ensemble_classifier.fit(X_train_processed, y_train)
    print("  Clasificador de Votación entrenado.")

    # Realizar predicciones en los datos de prueba preprocesados usando el Voting Classifier entrenado.
    print("  Evaluando Clasificador de Votación...")
    y_pred_ensemble = ensemble_classifier.predict(X_test_processed)

    # Evaluar el rendimiento del Voting Classifier.
    accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
    report_ensemble = classification_report(y_test, y_pred_ensemble, output_dict=True)

    print(f"    Precisión del Clasificador de Votación: {accuracy_ensemble:.4f}")

    # Almacenar las métricas de rendimiento del Voting Classifier en el diccionario de resultados.
    results[test_size]['Voting Classifier'] = {
        'accuracy': accuracy_ensemble,
        'report': report_ensemble
    }
    print("  Clasificador de Votación evaluado y resultados almacenados.")

In [38]:
# Initialize a dictionary to store the performance metrics for each split ratio.
# Inicializar un diccionario para almacenar las métricas de rendimiento para cada proporción de división.
results = {}

# Start a loop that iterates through each value in the split_ratios list.
# Iniciar un bucle que itera a través de cada valor en la lista split_ratios.
for test_size in split_ratios:
    # Print a message indicating the current split ratio being processed.
    # Imprimir un mensaje indicando la proporción de división actual que se está procesando.
    print(f"\nProcesando proporción de división (tamaño del conjunto de prueba): {test_size}")

    # Split the data into training and testing sets based on the current split ratio.
    # Dividir los datos en conjuntos de entrenamiento y prueba basándose en la proporción de división actual.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

    # Preprocesar las características: Convertir variables categóricas a numéricas usando One-Hot Encoding
    # Aplicar a los conjuntos de entrenamiento y prueba por separado para evitar fuga de datos
    X_train_processed = pd.get_dummies(X_train)
    X_test_processed = pd.get_dummies(X_test)

    # Asegurarse de que los conjuntos de entrenamiento y prueba tengan las mismas columnas después del one-hot encoding
    # Esto maneja casos donde una categoría puede aparecer en el test set pero no en el training set (o viceversa)
    X_train_processed, X_test_processed = X_train_processed.align(X_test_processed, join='left', axis=1, fill_value=0)

    print(f"  Datos divididos y preprocesados para tamaño de prueba {test_size}.")
    print(f"  Forma de X_train_processed: {X_train_processed.shape}")
    print(f"  Forma de X_test_processed: {X_test_processed.shape}")

    # Implementar y evaluar el clasificador de Árbol de Decisión.
    print("  Entrenando y evaluando Árbol de Decisión...")
    dt_classifier = DecisionTreeClassifier(random_state=42)
    dt_classifier.fit(X_train_processed, y_train)
    y_pred_dt = dt_classifier.predict(X_test_processed)
    accuracy_dt = accuracy_score(y_test, y_pred_dt)
    report_dt = classification_report(y_test, y_pred_dt, output_dict=True)
    print(f"    Precisión del Árbol de Decisión: {accuracy_dt:.4f}")

    # Store Decision Tree results
    # Almacenar resultados del Árbol de Decisión
    if test_size not in results:
        results[test_size] = {}
    results[test_size]['Decision Tree'] = {
        'accuracy': accuracy_dt,
        'report': report_dt
    }

    # Implementar y evaluar el clasificador k-NN.
    print("  Entrenando y evaluando k-NN...")
    # Using the best k found in the previous single split analysis (k=25)
    # Usando el mejor k encontrado en el análisis de división única anterior (k=25)
    knn_classifier = KNeighborsClassifier(n_neighbors=best_k)
    knn_classifier.fit(X_train_processed, y_train)
    y_pred_knn = knn_classifier.predict(X_test_processed)
    accuracy_knn = accuracy_score(y_test, y_pred_knn)
    report_knn = classification_report(y_test, y_pred_knn, output_dict=True)
    print(f"    Precisión de k-NN: {accuracy_knn:.4f}")

    # Store k-NN results
    # Almacenar resultados de k-NN
    results[test_size]['k-NN'] = {
        'accuracy': accuracy_knn,
        'report': report_knn
    }

    # Seleccionar las características para el modelo de Regresión Logística
    best_rule_pair = ('education-num', 'capital-gain')
    X_train_rule = X_train_processed[list(best_rule_pair)]
    X_test_rule = X_test_processed[list(best_rule_pair)]

    # Inicializar y entrenar el modelo de Regresión Logística
    print("  Entrenando y evaluando Regresión Logística (en características seleccionadas)...")
    lr_classifier = LogisticRegression(random_state=42)
    lr_classifier.fit(X_train_rule, y_train)

    # Realizar predicciones en el conjunto de prueba
    # Make predictions on the test set
    y_pred_lr = lr_classifier.predict(X_test_rule)

    # Evaluar el clasificador
    # Evaluate the classifier
    accuracy_lr = accuracy_score(y_test, y_pred_lr)
    report_lr = classification_report(y_test, y_pred_lr, output_dict=True)
    print(f"    Precisión de Regresión Logística: {accuracy_lr:.4f}")

    # Store Logistic Regression results
    # Almacenar resultados de Regresión Logística
    results[test_size]['Logistic Regression'] = {
        'accuracy': accuracy_lr,
        'report': report_lr
    }

    # Crear una lista de estimadores para el Voting Classifier.
    # Usar los modelos ya entrenados de la iteración de división actual.
    # Use the already trained models from the current split iteration.
    estimators = [
        ('dt', dt_classifier),
        ('knn', knn_classifier),
        ('lr', lr_classifier) # Usar el clasificador LR entrenado en características seleccionadas
    ]

    # Instantiate a VotingClassifier object with 'hard' voting.
    # n_jobs=-1 uses all available CPU cores for parallel processing, which can speed up training
    # Instanciar un objeto VotingClassifier con votación 'hard'.
    # n_jobs=-1 usa todos los núcleos de CPU disponibles para procesamiento paralelo, lo que puede acelerar el entrenamiento
    ensemble_classifier = VotingClassifier(estimators=estimators, voting='hard', n_jobs=-1)

    # Train the VotingClassifier on the preprocessed training data.
    # Entrenar el VotingClassifier en los datos de entrenamiento preprocesados.
    print("  Entrenando Clasificador de Votación...")
    ensemble_classifier.fit(X_train_processed, y_train)
    print("  Clasificador de Votación entrenado.")

    # Make predictions on the preprocessed test data using the trained Voting Classifier.
    # Realizar predicciones en los datos de prueba preprocesados usando el Voting Classifier entrenado.
    print("  Evaluando Clasificador de Votación...")
    y_pred_ensemble = ensemble_classifier.predict(X_test_processed)

    # Evaluate the performance of the Voting Classifier.
    # Evaluar el rendimiento del Voting Classifier.
    accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
    report_ensemble = classification_report(y_test, y_pred_ensemble, output_dict=True)

    print(f"    Precisión del Clasificador de Votación: {accuracy_ensemble:.4f}")

    # Store the performance metrics of the Voting Classifier in the results dictionary.
    # Almacenar las métricas de rendimiento del Voting Classifier en el diccionario de resultados.
    results[test_size]['Voting Classifier'] = {
        'accuracy': accuracy_ensemble,
        'report': report_ensemble
    }
    print("  Clasificador de Votación evaluado y resultados almacenados.")

In [39]:
# Initialize a dictionary to store the performance metrics for each split ratio.
results = {}

# Start a loop that iterates through each value in the split_ratios list.
for test_size in split_ratios:
    # Print a message indicating the current split ratio being processed.
    print(f"\nProcesando proporción de división (tamaño del conjunto de prueba): {test_size}")

    # Split the data into training and testing sets based on the current split ratio.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

    # Preprocesar las características: Convertir variables categóricas a numéricas usando One-Hot Encoding
    # Aplicar a los conjuntos de entrenamiento y prueba por separado para evitar fuga de datos
    X_train_processed = pd.get_dummies(X_train)
    X_test_processed = pd.get_dummies(X_test)

    # Asegurarse de que los conjuntos de entrenamiento y prueba tengan las mismas columnas después del one-hot encoding
    # Esto maneja casos donde una categoría puede aparecer en el test set pero no en el training set (o viceversa)
    X_train_processed, X_test_processed = X_train_processed.align(X_test_processed, join='left', axis=1, fill_value=0)

    print(f"  Datos divididos y preprocesados para tamaño de prueba {test_size}.")
    print(f"  Forma de X_train_processed: {X_train_processed.shape}")
    print(f"  Forma de X_test_processed: {X_test_processed.shape}")

    # Implementar y evaluar el clasificador de Árbol de Decisión.
    print("  Entrenando y evaluando Árbol de Decisión...")
    dt_classifier = DecisionTreeClassifier(random_state=42)
    dt_classifier.fit(X_train_processed, y_train)
    y_pred_dt = dt_classifier.predict(X_test_processed)
    accuracy_dt = accuracy_score(y_test, y_pred_dt)
    report_dt = classification_report(y_test, y_pred_dt, output_dict=True)
    print(f"    Precisión del Árbol de Decisión: {accuracy_dt:.4f}")

    # Almacenar resultados del Árbol de Decisión
    if test_size not in results:
        results[test_size] = {}
    results[test_size]['Decision Tree'] = {
        'accuracy': accuracy_dt,
        'report': report_dt
    }

    # Implementar y evaluar el clasificador k-NN.
    print("  Entrenando y evaluando k-NN...")
    # Usando el mejor k encontrado en el análisis de división única anterior (k=25)
    knn_classifier = KNeighborsClassifier(n_neighbors=best_k)
    knn_classifier.fit(X_train_processed, y_train)
    y_pred_knn = knn_classifier.predict(X_test_processed)
    accuracy_knn = accuracy_score(y_test, y_pred_knn)
    report_knn = classification_report(y_test, y_pred_knn, output_dict=True)
    print(f"    Precisión de k-NN: {accuracy_knn:.4f}")

    # Almacenar resultados de k-NN
    results[test_size]['k-NN'] = {
        'accuracy': accuracy_knn,
        'report': report_knn
    }

    # Seleccionar las características para el modelo de Regresión Logística
    best_rule_pair = ('education-num', 'capital-gain')
    X_train_rule = X_train_processed[list(best_rule_pair)]
    X_test_rule = X_test_processed[list(best_rule_pair)]

    # Inicializar y entrenar el modelo de Regresión Logística
    print("  Entrenando y evaluando Regresión Logística (en características seleccionadas)...")
    lr_classifier = LogisticRegression(random_state=42)
    lr_classifier.fit(X_train_rule, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_pred_lr = lr_classifier.predict(X_test_rule)

    # Evaluar el clasificador
    accuracy_lr = accuracy_score(y_test, y_pred_lr)
    report_lr = classification_report(y_test, y_pred_lr, output_dict=True)
    print(f"    Precisión de Regresión Logística: {accuracy_lr:.4f}")

    # Almacenar resultados de Regresión Logística
    results[test_size]['Logistic Regression'] = {
        'accuracy': accuracy_lr,
        'report': report_lr
    }

    # Crear una lista de estimadores para el Voting Classifier.
    # Usar los modelos ya entrenados de la iteración de división actual.
    estimators = [
        ('dt', dt_classifier),
        ('knn', knn_classifier),
        ('lr', lr_classifier) # Usar el clasificador LR entrenado en características seleccionadas
    ]

    # Instanciar un objeto VotingClassifier con votación 'hard'.
    # n_jobs=-1 usa todos los núcleos de CPU disponibles para procesamiento paralelo, lo que puede acelerar el entrenamiento
    ensemble_classifier = VotingClassifier(estimators=estimators, voting='hard', n_jobs=-1)

    # Entrenar el VotingClassifier en los datos de entrenamiento preprocesados.
    print("  Entrenando Clasificador de Votación...")
    ensemble_classifier.fit(X_train_processed, y_train)
    print("  Clasificador de Votación entrenado.")

    # Realizar predicciones en los datos de prueba preprocesados usando el Voting Classifier entrenado.
    print("  Evaluando Clasificador de Votación...")
    y_pred_ensemble = ensemble_classifier.predict(X_test_processed)

    # Evaluar el rendimiento del Voting Classifier.
    accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
    report_ensemble = classification_report(y_test, y_pred_ensemble, output_dict=True)

    print(f"    Precisión del Clasificador de Votación: {accuracy_ensemble:.4f}")

    # Almacenar las métricas de rendimiento del Voting Classifier en el diccionario de resultados.
    results[test_size]['Voting Classifier'] = {
        'accuracy': accuracy_ensemble,
        'report': report_ensemble
    }
    print("  Clasificador de Votación evaluado y resultados almacenados.")

In [40]:
# Definir los diferentes tamaños del conjunto de prueba para las divisiones de entrenamiento-prueba.
# Esto corresponde a tamaños de entrenamiento del 60%, 40%, 20% y 10% respectivamente.
split_ratios = [0.6, 0.4, 0.2, 0.1]

print("Proporciones de división entrenamiento-prueba (tamaño del conjunto de prueba):")
print(split_ratios)

# Inicializar un diccionario para almacenar las métricas de rendimiento para cada proporción de división.
results = {}

# Iniciar un bucle que itera a través de cada valor en la lista split_ratios.
for test_size in split_ratios:
    # Imprimir un mensaje indicando la proporción de división actual que se está procesando.
    print(f"\nProcesando proporción de división (tamaño del conjunto de prueba): {test_size}")

    # Dividir los datos en conjuntos de entrenamiento y prueba basados en la proporción de división actual.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

    # Preprocesar las características: Convertir variables categóricas a numéricas usando One-Hot Encoding
    # Aplicar a los conjuntos de entrenamiento y prueba por separado para evitar fuga de datos (data leakage)
    X_train_processed = pd.get_dummies(X_train)
    X_test_processed = pd.get_dummies(X_test)

    # Asegurar que los conjuntos de entrenamiento y prueba tengan las mismas columnas después del one-hot encoding
    # Esto maneja casos donde una categoría puede aparecer en el conjunto de prueba pero no en el de entrenamiento (o viceversa)
    X_train_processed, X_test_processed = X_train_processed.align(X_test_processed, join='left', axis=1, fill_value=0)

    print(f"  Datos divididos y preprocesados para tamaño de prueba {test_size}.")
    print(f"  Forma de X_train_processed: {X_train_processed.shape}")
    print(f"  Forma de X_test_processed: {X_test_processed.shape}")

    # Implementar y evaluar el clasificador de Árbol de Decisión.
    print("  Entrenando y evaluando Árbol de Decisión...")
    dt_classifier = DecisionTreeClassifier(random_state=42)
    dt_classifier.fit(X_train_processed, y_train)
    y_pred_dt = dt_classifier.predict(X_test_processed)
    accuracy_dt = accuracy_score(y_test, y_pred_dt)
    report_dt = classification_report(y_test, y_pred_dt, output_dict=True)
    print(f"    Precisión del Árbol de Decisión: {accuracy_dt:.4f}")

    # Almacenar resultados del Árbol de Decisión
    if test_size not in results:
        results[test_size] = {}
    results[test_size]['Decision Tree'] = {
        'accuracy': accuracy_dt,
        'report': report_dt
    }

    # Implementar y evaluar el clasificador k-NN.
    print("  Entrenando y evaluando k-NN...")
    # Usando el mejor k encontrado en el análisis de división única anterior (k=25)
    knn_classifier = KNeighborsClassifier(n_neighbors=best_k)
    knn_classifier.fit(X_train_processed, y_train)
    y_pred_knn = knn_classifier.predict(X_test_processed)
    accuracy_knn = accuracy_score(y_test, y_pred_knn)
    report_knn = classification_report(y_test, y_pred_knn, output_dict=True)
    print(f"    Precisión de k-NN: {accuracy_knn:.4f}")

    # Almacenar resultados de k-NN
    results[test_size]['k-NN'] = {
        'accuracy': accuracy_knn,
        'report': report_knn
    }

    # Seleccionar las características para el modelo de Regresión Logística
    best_rule_pair = ('education-num', 'capital-gain')
    X_train_rule = X_train_processed[list(best_rule_pair)]
    X_test_rule = X_test_processed[list(best_rule_pair)]

    # Inicializar y entrenar el modelo de Regresión Logística
    print("  Entrenando y evaluando Regresión Logística (en características seleccionadas)...")
    lr_classifier = LogisticRegression(random_state=42)
    lr_classifier.fit(X_train_rule, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_pred_lr = lr_classifier.predict(X_test_rule)

    # Evaluar el clasificador
    accuracy_lr = accuracy_score(y_test, y_pred_lr)
    report_lr = classification_report(y_test, y_pred_lr, output_dict=True)
    print(f"    Precisión de Regresión Logística: {accuracy_lr:.4f}")

    # Almacenar resultados de Regresión Logística
    results[test_size]['Logistic Regression'] = {
        'accuracy': accuracy_lr,
        'report': report_lr
    }

    # Crear una lista de estimadores para el Voting Classifier.
    # Usar los modelos ya entrenados de la iteración de división actual.
    estimators = [
        ('dt', dt_classifier),
        ('knn', knn_classifier),
        ('lr', lr_classifier) # Usar el clasificador LR entrenado en características seleccionadas
    ]

    # Instanciar un objeto VotingClassifier con votación 'hard'.
    # n_jobs=-1 usa todos los núcleos de CPU disponibles para procesamiento paralelo, lo que puede acelerar el entrenamiento
    ensemble_classifier = VotingClassifier(estimators=estimators, voting='hard', n_jobs=-1)

    # Entrenar el VotingClassifier en los datos de entrenamiento preprocesados.
    print("  Entrenando Clasificador de Votación...")
    # Es importante notar que el VotingClassifier espera que todos los estimadores
    # en 'estimators' acepten el mismo formato de entrada. Dado que lr_classifier
    # fue entrenado solo con un subconjunto de columnas, esto podría causar un error
    # si el VotingClassifier intenta pasarle X_train_processed completo.
    # Para una implementación más robusta, se debería usar un Pipeline con ColumnTransformer
    # para el modelo de Regresión Logística dentro del ensemble.
    # Sin embargo, para cumplir con el propósito de este análisis comparativo simple,
    # y asumiendo que los modelos pueden manejar las columnas que necesitan, procedemos.
    # Nota: En un escenario de producción, se recomienda usar Pipelines.

    # Para que el VotingClassifier funcione correctamente aquí, debemos entrenar
    # los modelos individuales (DT, k-NN) con X_train_processed completo, y el modelo
    # de Regresión Logística con X_train_rule (el subconjunto).
    # Luego, al crear el VotingClassifier, si no usamos Pipelines, solo podemos
    # incluir modelos que acepten X_train_processed completo.
    # Esto significa que el modelo de Regresión Logística debe ser adaptado o
    # re-entrenado para aceptar X_train_processed completo y seleccionar sus columnas internamente,
    # o usamos un Pipeline como se hizo en la celda anterior (Q7.1).

    # Vamos a usar la estrategia del Pipeline para el modelo de Regresión Logística dentro del ensemble
    # para asegurar que reciba solo las columnas que necesita, mientras que el ensemble
    # recibe X_train_processed completo.

    # Re-inicializar los modelos individuales para asegurar que se entrenen con los datos del split actual
    dt_classifier_ensemble = DecisionTreeClassifier(random_state=42)
    knn_classifier_ensemble = KNeighborsClassifier(n_neighbors=best_k)
    lr_classifier_ensemble = LogisticRegression(random_state=42)

    # Entrenar los modelos individuales con los datos del split actual
    dt_classifier_ensemble.fit(X_train_processed, y_train)
    knn_classifier_ensemble.fit(X_train_processed, y_train)
    # Entrenar el modelo de Regresión Logística con el subconjunto de columnas para este split
    lr_classifier_ensemble.fit(X_train_rule, y_train)


    # Crear un Pipeline para el modelo de reglas que primero selecciona las columnas
    rule_pipeline = Pipeline([
        ('selector', ColumnTransformer([('passthrough', 'passthrough', list(best_rule_pair))],
                                      remainder='drop')), # Selecciona solo las columnas del par
        ('model', lr_classifier_ensemble) # Usar el modelo LR entrenado para este split
    ])

    # Actualizar la lista de estimadores con los modelos entrenados y el pipeline de reglas
    estimators_ensemble_piped = [
        ('dt', dt_classifier_ensemble),
        ('knn', knn_classifier_ensemble),
        ('rule_pipe', rule_pipeline) # Pipeline de reglas para este split
    ]

    # Inicializar el VotingClassifier con los modelos entrenados y el pipeline de reglas para este split
    ensemble_classifier = VotingClassifier(estimators=estimators_ensemble_piped, voting='hard', n_jobs=-1)


    # Entrenar el clasificador de conjunto con los datos de entrenamiento preprocesados para este split
    print("  Entrenando Clasificador de Votación (con Pipelines)...")
    ensemble_classifier.fit(X_train_processed, y_train)
    print("  Clasificador de Votación entrenado.")

    # Realizar predicciones en los datos de prueba preprocesados usando el Voting Classifier entrenado.
    print("  Evaluando Clasificador de Votación...")
    y_pred_ensemble = ensemble_classifier.predict(X_test_processed)

    # Evaluar el rendimiento del Voting Classifier.
    accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
    report_ensemble = classification_report(y_test, y_pred_ensemble, output_dict=True)

    print(f"    Precisión del Clasificador de Votación: {accuracy_ensemble:.4f}")

    # Almacenar las métricas de rendimiento del Voting Classifier en el diccionario de resultados.
    results[test_size]['Voting Classifier'] = {
        'accuracy': accuracy_ensemble,
        'report': report_ensemble
    }
    print("  Clasificador de Votación evaluado y resultados almacenados.")

# Imprimir un resumen de las precisiones obtenidas para cada modelo en cada división.
print("\n--- Resumen de Precisión en Diferentes Divisiones Entrenamiento-Prueba ---")

# Crear un diccionario para contener las precisiones para facilitar la creación del DataFrame
accuracy_summary = {}
for test_size, models in results.items():
    accuracy_summary[test_size] = {model_name: model_results['accuracy'] for model_name, model_results in models.items()}

# Crear un DataFrame a partir del diccionario de resumen de precisión
# Transponer el DataFrame para que los modelos sean las filas y los tamaños de prueba sean las columnas
accuracy_df = pd.DataFrame.from_dict(accuracy_summary, orient='index').transpose()

# Ordenar las columnas por tamaño de prueba (ascendente) para una mejor legibilidad
accuracy_df = accuracy_df.sort_index(axis=1)

print("Resumen de Precisión a Través de Diferentes Divisiones Entrenamiento-Prueba:")
display(accuracy_df)

# (Opcional) Crear un gráfico de líneas mostrando la precisión de cada modelo a través de las diferentes proporciones de división.
plt.figure(figsize=(12, 7))
sns.lineplot(data=accuracy_df.transpose(), marker='o') # Transponer de nuevo para graficar test_size en el eje x
plt.title('Precisión del Modelo vs. Proporción de División Entrenamiento-Prueba')
plt.xlabel('Tamaño del Conjunto de Prueba (Proporción de División)')
plt.ylabel('Precisión (Accuracy)')
plt.xticks(accuracy_df.columns) # Establecer las marcas del eje x a los tamaños de prueba
plt.grid(True)
plt.legend(title='Modelo')
plt.tight_layout() # Ajustar el diseño

# (Opcional) Guardar el gráfico en el directorio de figuras definido por la variable figs.
plot_path = os.path.join(figs, "model_accuracy_vs_split_ratio.png")
plt.savefig(plot_path, dpi=300)
print(f"Gráfico de precisión guardado en: {plot_path}")

# (Opcional) Mostrar el gráfico.
plt.show()

In [41]:
'''
Guardar en mi drive los notebooks en formato .ipynb y .py.
'''

notebook_name = "notebook_taller3"

# Ruta completa de salida en Drive
ipynb_path = os.path.join(codes, f"{notebook_name}.ipynb")
py_path = os.path.join(codes, f"{notebook_name}.py")

# Ruta temporal para guardar el notebook actual antes de convertir
temp_ipynb_path = "/content/temp_notebook.ipynb"

# Guardar el notebook actual a una ruta temporal usando el comando mágico %notebook
# Esto asegura que nbconvert tenga un archivo .ipynb específico para trabajar
get_ipython().run_line_magic('notebook', temp_ipynb_path)


# Guardar en Drive en formato .ipynb
# Usar el archivo temporal guardado por %notebook
!jupyter nbconvert --to notebook --output "{ipynb_path}" "{temp_ipynb_path}"

# Guardar en Drive en formato .py
# Usar el archivo temporal guardado por %notebook
!jupyter nbconvert --to script   --output "{py_path}"    "{temp_ipynb_path}"

print(f"Archivos guardados en Drive:\n- {ipynb_path}\n- {py_path}")